In [1]:
from discussion_agents.cog.agent.expel import ExpeLAgent

c:\Users\tuvin\anaconda3\envs\discussion-agents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
agent = ExpeLAgent()

In [3]:
agent.experience_memory

In [4]:
agent.insight_memory